# Datatón Bancolombia 2020

### Librerias

In [1]:
#libreria | objetivo

#procesamiento
import pandas as pd   #manejo de dataframes
from dask import dataframe as dd #pandas + numpy + sklearn con paralelismo

#matematicas, estadistica y graficas
import numpy as np #numpy para funciones matematicas
import matplotlib.pyplot as plt #para graficar
import seaborn as sns #para graficar
import matplotlib.ticker as mtick #para ajustar estilo de ejes 
from scipy.stats import median_absolute_deviation #para calcular la MAD
from sklearn.model_selection import train_test_split #para el random split de los datos
from sklearn.preprocessing import StandardScaler  #para el scaler de las variables
from sklearn.decomposition import PCA #para hacer componentes principales

#Modelos

#supervizados
from sklearn.ensemble import RandomForestRegressor #para hacer un RandomForestRegressor
from sklearn.linear_model import Ridge #regresion ridge
from sklearn.svm import SVR #regresion SVM
from sklearn.neighbors import KNeighborsRegressor #regresion KNN
from sklearn.ensemble import GradientBoostingRegressor #regresion Gradient Boosting
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

#no supervizados
from sklearn.cluster import KMeans #k-means

### Configuración del notebook

In [2]:
%config IPCompleter.greedy = True #para que el notebook autocomplete
plt.style.use('Solarize_Light2') #ajustar estilo de matplotlib  Solarize_Light2

## Preprocesamiento

### Funciones

In [3]:
def fuck_comas(df):  
    
    '''Soluciona el problema con las profesiones que en su valor tienen comas, haciendo que la lectura del csv falle.
    Parte de que a la lectura del DataFrame se le añadieron dos columnas ficticias (col1 y col2) para manejar el desplazamiento.'''
    
    df.iloc[df.loc[df.col2.notnull()].index, 7] = df.iloc[df.loc[df.col2.notnull()].index, 7] + df.iloc[df.loc[df.col2.notnull()].index, 8] + df.iloc[df.loc[df.col2.notnull()].index, 9] #la columna 7 es concatenada con valores que se encuentran hasta dos columnas mas adelante
    df.iloc[df.loc[df.col2.notnull()].index, 8:65] = df.iloc[df.loc[df.col2.notnull()].index, 10:67].to_numpy() #las columnas posteriores a la 7 son llenadas con los valores con los valores que se encuentran desplazadas dos veces
    
    df.iloc[df.loc[df.col2.notnull()].index, 65:67] = float('nan')  #las columnas dummies col1 y col2 se llenan de nan
    df.drop(columns = ['col2'], inplace = True) #se elimina la columna col2
    
    df.iloc[df.loc[df.col1.notnull()].index, 7] = df.iloc[df.loc[df.col1.notnull()].index, 7] + df.iloc[df.loc[df.col1.notnull()].index, 8] #la columna 7 es concatenada con un valor desplazado una columna para solucionar lo de las profesiones que solo desplazaron los datos una vez
    df.iloc[df.loc[df.col1.notnull()].index, 8:65] = df.iloc[df.loc[df.col1.notnull()].index, 9:66].to_numpy() #las columnas posteriores a la 7 se llenan con los datos que solo se desplazaron una vez
    df.drop(columns = ['col1'], inplace = True) #se elimina la columna col1
    
    return df

def dtype_errors(df):
    
    '''Imprime los valores unicos de todas las columnas de un DataFrame, posibilitanto encontrar valores que se encuentres desplazados'''
    
    type_errors = {df.columns[i]: df[df.columns[i]].unique() for i in range(0, len(df.columns))}  #trae los valores unicos de cada columna
    
    print(type_errors)
    
def t_ult_actual_y_covid(df):
    
    '''Reemplaza la columna ult_actual del dataframe original por la columna t_ult_actual, 
    que corresponde a la cantidad de meses entre periodo y ult_actual. Adicionalmente, elimina los registros donde la 
    variable periodo y ult_actual son nulos, ademas de eliminar los que no continen el str(2) porque entonces no 
    podrian ser fechas del siglo XXI, ademas de filtrar por la cantidad de caracteres.
    
    A su vez, esta funcion verifica el valor de la columna periodo para clasificar por precovid y postcovid'''
    
    df = df[df.periodo.notnull()] #elimina los registros donde la variable periodo y ult_actual son nulos
    df = df[df.ult_actual.notnull()] 

    df = df[df.periodo.str.contains("2")]   #elimina los que no continen el str(2) porque entonces no podrian ser fechas del siglo XXI
    df = df[df.ult_actual.str.contains("2")] 

    df = df[df['ult_actual'].map(len) == 8]  #eliminamos todos los registros de esta variable que no tienen 8 de longitud
    df = df[df['periodo'].map(len) == 6]   ##eliminamos todos los registros de esta variable que no tienen 6 de longitud

    periodo = [[int(x[:4])*12, int(x[4:])] for x in df.periodo] #lista de listas del periodo partido en [año, mes] donde el año se ha convertido a meses
    ult_actual = [[int(x[:4])*12, int(x[4:6])] for x in df.ult_actual] #lista de listas del ult_actual partido en [año, mes] donde el año se ha convertido a meses
    a = [periodo[i][0] - ult_actual[i][0] + (periodo[i][1] - ult_actual[i][1]) for i in range(0,len(periodo))] #equivalente a un np.sum(dataframe, axis =1) para sumar los años convertidos y los meses y obtener el tiempo total
    
    #los sgtes pasos hicieron parte del codigo, pero numpy tiene muchos conflictos con dask, entonces lo que hacen las 
    #lineas siguientes ahora es hecho sin numpy por las tres lineas anteriores.
    #a = np.array(periodo) - np.array(ult_actual) #elementwise diferencia
    #a[:, 0] = a[:, 0] * 12 #conversion de años a meses
    #a = np.sum(a, axis = 1) #suma por axis = 1, para que sea por columna
    
    df['ult_actual'] = a #redefinicion de ult_actual
    df = df.rename(columns = {'ult_actual': 't_ult_actual'}) #cambio de nombre
    
    #lo siguiente toma el valor de la columna periodo para clasificar por precovid y postcovid
    #era una funcion aparte de t_ult_actual pero como obvio los filtros que estan al inicio de t_ult_actual
    #no se han computado cuando se declara la funcion covid en el map_partitions, entonces se tienen los 
    #mismos errores de preprocesamiento, es decir, campos que no se pueden convertir a enteros, etc.
    #para ahorrar CPU y RAM se prefiere implementarla aca mismo
     
    df['mes'] = [int(i[-2:]) for i in df.periodo] #para capturar el numero del mes
    df.mes = df.mes.replace(to_replace = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], value = ['enero', 'febrero', 'marzo', 
                                                                                          'abril', 'mayo', 'junio', 
                                                                                          'julio', 'agosto', 'septiembre', 
                                                                                          'octubre', 'noviembre', 'diciembre']) #para poner el nombre del mes
    
    df['periodo'] = [1 if int(i) >= 202003 else 0 for i in df.periodo] #redefinicion de periodo para tener el flag de covid
    df = df.rename(columns = {'periodo': 'covid'}) #cambio de nombre    
    
    return df

def edad_to_float(df):
    
    '''convierte la columna edad a un flotante y reemplaza \\N por la mediana.
    Tiene sentido imputar porque los NA son cerca de 0.03%'''
    
    df.edad = df.edad.replace(to_replace = ['\\N','foo'], value = [float('nan'), float('nan')]) #reemplaza los '\\N' de edad como nan, tambien el 'foo'
    df['edad'] = [float(i) for i in df['edad']]  #convierte todo a flotante
    edad_median = df.edad.median()  #calcula la mediana 
    df['edad'] = df.edad.fillna(edad_median)  #reemplaza la mediana
    
    return df

def genero_limpio(df):
    
    '''imputa el valor ' ' de la columna genero por la moda, tiene sentido imputar asi porque hay pocos ' ' '''
    
    df.genero = df.genero.replace(to_replace = ' ', value = float('nan')) #reemplaza los ' ' de genero como nan
    genero_mode = df.genero.mode().iloc[0]  #calcula la moda 
    df['genero'] = df.genero.fillna(genero_mode)  #reemplaza la moda
    
    return df

def estado_civil_limpio(df):
    
    '''imputa el valor de \\N de la columna estado_civil por la moda 
    tiene sentido porque son pocos los que no reportan'''
    
    df.estado_civil = df.estado_civil.replace(to_replace = '\\N', value = float('nan')) #reemplaza los '\\N' de estado civil como nan
    estado_civil_mode = df.estado_civil.mode().iloc[0]  #calcula la moda 
    df['estado_civil'] = df.estado_civil.fillna(estado_civil_mode)  #reemplaza la moda
    
    return df

def nivel_academico_limpio(df):
    
    '''Reemplaza el valor NO INFORMA por SIN INFORMACION, asumimos que no tiene sentido imputador en esta variable'''
    
    df.nivel_academico = df.nivel_academico.replace(to_replace = 'NO INFORMA', value = 'SIN INFORMACION') #reemplaza los 'NO INFORMA' de genero como 'SIN INFORMACION'
    
    return df

def profesion_limpia(df):
    
    '''Elimina la variable profesion. No se deja en el modelo porque la matriz seria muy dispersa, sin embargo, 
    luego podrian agruparse las profesiones por categorias similares (si se quiere)'''
    
    df.drop('profesion', inplace = True, axis=1) #elimina la columna profesion
    
    return df
    
    
def ocupacion_limpia(df):
    
    '''Imputa en ocupacion y corrige los faltantes'''
    
    df.ocupacion = df.ocupacion.replace(to_replace = ['Profesional Independiente' , '\\N', 'Jubilado', 'Vacío'], 
                                        value = ['Independiente', 'Sin Ocupacion Asignada', 'Pensionado', 'Sin Ocupacion Asignada'])
    
    return df

def tipo_vivienda_limpia(df):

    df.tipo_vivienda = df.tipo_vivienda.replace(to_replace = ['\\N'], 
                                    value = ['NO INFORMA'])
    
    return df
    
def categoria_limpio(df):
    
    '''imputa el valor de \\N de la columna categoria por la moda SIN EMBARGO, SERIE INTERESANTE ENTRENAR UN CLASIFICADOR
    TIPO REGRESION LOGISTICA PARA SU CALCULO'''
    
    '''TAMBIEN SERIE INTERESANTE VER SI LA MODA SE REEMPLAZA POR PARTICION O POR TODA LA BASE DE DATOS'''
    
    df.categoria = df.categoria.replace(to_replace = '\\N', value = float('nan')) #reemplaza los '\\N' de categoria como nan
    categoria_mode = df.categoria.mode().iloc[0]  #calcula la moda 
    df['categoria'] = df.categoria.fillna(categoria_mode)  #reemplaza la moda
    
    return df

def ind_binarios_riesgo(df):
    
    '''Imputa los valores "\\N" por "N" puesto que si no hay informacion lo mas normal seria que no tenga (el banco 
    puede que no sepa si no tienes moras o carteras cast. pero si sabe cuando si).
    luego convierte los "S" en 1 y los "N" en 0.
    Aplica para: ind_mora_vigente, cartera_castigada'''

    df.ind_mora_vigente = df.ind_mora_vigente.replace(to_replace = ['\\N','N','foo','S'], 
                                    value = ['0','0','0','1'])
    df.cartera_castigada = df.cartera_castigada.replace(to_replace = ['\\N','N','foo','S'], 
                                    value = ['0','0','0','1'])
    df.ind_mora_vigente = pd.to_numeric(df.ind_mora_vigente)
    df.cartera_castigada = pd.to_numeric(df.cartera_castigada)

    return df

def rechazo_credito_limpio(df):
    
    '''Elimina la variable rechazo_credito porque el 99% son datos faltantes. Antes del análisis de nulos esta funcion hacia 
    lo sgte: 
    
        Imputa los valores "\\N" por "No Rechazos" puesto que al tratarse de un flag de rechazo si no
        aparece valor es por que no ha tenido rechazos
        
            df.rechazo_credito = df.rechazo_credito.replace(to_replace = ['\\N'], 
                                    value = ['No Rechazos'])
    '''
    
    df = df.drop(columns = ['rechazo_credito'])

    return df

def manejo_de_moras(df):
    
    '''Vamos a atacar las moras en conjunto bajo la misma hipotesis de los indicadores de riesgo si no hay info
    es que no hay, el banco siempre sabe cuando si (sino para que putas pagaria datacredito). Igualmente para blindarnos
    revisamos si tiene flag de moras de mas de 30, 60 o 90 en los ultimos 12 meses en alguno de los valores vacios de
    mora max y no fue el caso. Ademas tambien comprobamos que donde hay valores de mora max, hay reportes para los indicadores
    de moras de x dias correspondientes'''
    
    #no vi el foo en las primeras 8 particiones pero como me sacó error entonces se los meti a todos por silas
    df.mora_max = df.mora_max.replace(to_replace = ['\\N','foo', float('nan')], 
                                    value = ['0','0', '0'])
    
    df.cant_moras_30_ult_12_meses = df.cant_moras_30_ult_12_meses.replace(to_replace = ['\\N','foo', float('nan')], 
                                    value = ['0','0', '0'])
    
    df.cant_moras_60_ult_12_meses = df.cant_moras_60_ult_12_meses.replace(to_replace = ['\\N','foo', float('nan')], 
                                    value = ['0','0', '0'])
    
    df.cant_moras_90_ult_12_meses = df.cant_moras_90_ult_12_meses.replace(to_replace = ['\\N','foo', float('nan')], 
                                    value = ['0','0', '0'])
    
    #de una vez convirtamos todo esto en numeros para irnos acercando a los tipos de datos ideales
    df.mora_max = pd.to_numeric(df.mora_max)
    df.cant_moras_30_ult_12_meses = pd.to_numeric(df.cant_moras_30_ult_12_meses)
    df.cant_moras_60_ult_12_meses = pd.to_numeric(df.cant_moras_60_ult_12_meses)
    df.cant_moras_90_ult_12_meses = pd.to_numeric(df.cant_moras_90_ult_12_meses)
    
    return df

def limpieza_TdC(df):
    
    '''Apriori no parece requerir imputacion mas alla de los foo, que corresponden a falta de info y por consiguiente 
    deberian ser 0. De cualquier manera la mediana de estos valores corresponde a 0 pues 60% de estos datos ya estan en 0 
    y el foo es un valor ficticio explicado abajo'''
    
    df.cupo_total_tc = df.cupo_total_tc.replace(to_replace = ['foo', float('nan')], 
                                    value = ['0', '0'])
    df.cuota_tc_bancolombia = df.cuota_tc_bancolombia.replace(to_replace = ['foo', float('nan')], 
                                    value = ['0', '0'])
    
    df.cupo_total_tc = pd.to_numeric(df.cupo_total_tc)
    df.cuota_tc_bancolombia = pd.to_numeric(df.cuota_tc_bancolombia)
    
    return df

def limpieza_obligaciones_cred(df):
    
    '''Dado que solo hay "\\N" y "X" los primeros se reemplazaran por 0 y los segundos por 1 en las columnas
    de tiene_crediagil y tiene_consumo'''
    
    df.tiene_consumo = df.tiene_consumo.replace(to_replace = ['\\N', 'X', 'foo',float('nan')], 
                                    value = [0, 1, 0, 0])
    df.tiene_crediagil = df.tiene_crediagil.replace(to_replace = ['\\N', 'X', 'foo', float('nan')], 
                                    value = [0, 1, 0, 0])
    
    df.tiene_consumo = pd.to_numeric(df.tiene_consumo)
    df.tiene_crediagil = pd.to_numeric(df.tiene_crediagil)


    return df

def limpieza_cuentas(df):
    
    '''Para el numero de cuentas, cuentas embargas y actuvas,
    parece seguro imputar un valor de 0 a lo que no se tenga info. ademas se comprobó que estos \\N son paralelos'''
    
    df.nro_tot_cuentas = df.nro_tot_cuentas.replace(to_replace = ['\\N', 'foo', float('nan')], 
                                    value = ['0', '0', '0'])
    df.ctas_activas = df.ctas_activas.replace(to_replace = ['\\N', 'foo', float('nan')], 
                                    value = ['0', '0', '0'])
    df.ctas_embargadas = df.ctas_embargadas.replace(to_replace = ['\\N', 'foo', float('nan')], 
                                    value = ['0', '0', '0'])
    
    df.nro_tot_cuentas = pd.to_numeric(df.nro_tot_cuentas)
    df.ctas_activas = pd.to_numeric(df.ctas_activas)
    df.ctas_embargadas = pd.to_numeric(df.ctas_embargadas)
    
    return df

def limpieza_fopep(df):
    
    '''Vamos a reemplazar el foo y el \\N  por 0 y la X por 1.'''
    
    df.pension_fopep = df.pension_fopep.replace(to_replace = ['\\N', 'foo', 'X', float('nan')], 
                                    value = ['0', '0', "1", '0'])
    df.pension_fopep = pd.to_numeric(df.pension_fopep)
    
    return df

def limpieza_hipotecario(df):
    
    '''Vamos a limpiar lo concerniente a creditos hipotecarios.
    Que en todo caso seria para cuota_cred_hipot, tiene_cred_hipo_1, tiene_cred_hipo_2'''
    
    df.tiene_cred_hipo_1 = df.tiene_cred_hipo_1.replace(to_replace = [float('nan'),'\\N', 'foo', 'X'], 
                                                value = ['0', '0', '0', "1"])
    df.tiene_cred_hipo_2 = df.tiene_cred_hipo_2.replace(to_replace = [float('nan'),'\\N', 'foo', 'X'], 
                                            value = ['0', '0', '0', "1"])
    df.cuota_cred_hipot = df.cuota_cred_hipot.replace(to_replace = ['\\N', 'foo', float('nan')], 
                                            value = ['0', '0', '0'])
    
    df.cuota_cred_hipot = pd.to_numeric(df.cuota_cred_hipot)
    
    return df

def limpieza_mediana_nom3(df):
    
    '''Esta funcion transforma los datos de mediana_nom3 en un float. Imputa los foo por la mediana'''
    
    df.mediana_nom3 = df.mediana_nom3.replace(to_replace = ['foo'], value = [float('nan')])
    df['mediana_nom3'] = [float(i) for i in df['mediana_nom3']]  #convierte todo a flotante
    mediana_nom3_median = df.mediana_nom3.median()  #calcula la mediana 
    df['mediana_nom3'] = df.mediana_nom3.fillna(mediana_nom3_median) 
    
    return df

def limpieza_mediana_pen3(df):
    
    '''Esta funcion transforma los datos de mediana_pen3 en un float. Imputa los foo  por la mediana'''
    
    df.mediana_pen3 = df.mediana_pen3.replace(to_replace = ['foo'], value = [float('nan')])
    df['mediana_pen3'] = [float(i) for i in df['mediana_pen3']]  #convierte todo a flotante
    mediana_pen3_median = df.mediana_pen3.median()  #calcula la mediana 
    df['mediana_pen3'] = df.mediana_pen3.fillna(mediana_pen3_median) 
    
    return df

def limpieza_ingreso_nompen(df):
    
    '''Esta funcion transforma los datos de ingreso_nompen en un float. Imputa los foo  por la mediana'''
    
    df.ingreso_nompen = df.ingreso_nompen.replace(to_replace = ['foo'], value = [float('nan')])
    df['ingreso_nompen'] = [float(i) for i in df['ingreso_nompen']]  #convierte todo a flotante
    ingreso_nompen_median = df.ingreso_nompen.median()  #calcula la mediana 
    df['ingreso_nompen'] = df.ingreso_nompen.fillna(ingreso_nompen_median) 
    
    return df

def limpieza_cat_ingreso(df):
    
    '''Esta funcion transforma los datos de ingreso_nompen en un float. Imputa los foo  por la mediana'''
    
    df.cat_ingreso = df.cat_ingreso.replace(to_replace = ['\\N', float('nan')], value = ['Sin informacion', 'Sin informacion'])
    
    
    return df

def limpeza_al_estilo_datacredito(df):
    
    '''Funcion que toma todas las variables desde ingreso final hasta el ind y aprovecha que todas son numéricas para hacerles
    el mismo tratamiento: reemplazar foo's y \\N's por CEROS (ASUMUENDO QUE DATACREDITO SI TIENE ESA INFORMACION) 
    Y QUE SI APARECE \\NA ES PORQUE ES CERO.
    Nos soplamos cuota_tc_mdo porque es combinacion lineal de cuota_tarjeta_de_credito y cuota_tc_bancolombia'''
    
    df = df.drop(columns = ['cuota_tc_mdo']) 

    variables_double = ['cant_mora_30_tdc_ult_3m_sf', 'cant_mora_30_consum_ult_3m_sf',
   'cuota_de_vivienda', 'cuota_de_consumo', 'cuota_rotativos',
   'cuota_tarjeta_de_credito', 'cuota_de_sector_solidario',
   'cuota_sector_real_comercio', 'cupo_tc_mdo', 'saldo_prom3_tdc_mdo', 
    'saldo_no_rot_mdo', 'cuota_libranza_sf', 'cant_oblig_tot_sf', 'cant_cast_ult_12m_sr']
    
    for i in variables_double:
    
        df[i] = df[i].replace(to_replace = ['foo', '\\N', float('nan')], value = [0, 0, 0])
        df[i] = [float(i) for i in df[i]]  #convierte todo a flotante
    
    return df

def limpeza_ingresos_gastos(df):
    
    '''Esta funcion sirve para limpiar e imputar con la mediana las variables ingreso_final , ind y gasto_familiar.
    A su vez, elimina ingreso_nomina y ingreso_segurida_social por estar llenos de \\N'''
    
    df = df.drop(columns = ['ingreso_nomina', 'ingreso_segurida_social'])
    
    variables = ['ingreso_final', 'ind', 'gasto_familiar']
    
    for i in variables:
    
        df[i] = df[i].replace(to_replace = ['foo', '\\N'], value = [float('nan'), float('nan')])
        df[i] = [float(i) for i in df[i]]  #convierte todo a flotante
        i_median = df[i].median()  #calcula la mediana 
        df[i] = df[i].fillna(i_median) 
    
    return df

def arreglo_col_mes(df):
    
    '''Esta funcion simplemente mueve la ultima columna del df (mes) a la primera posicion'''
    
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df=df[cols]
    
    return df

def Limpieza_calif_cent_ext(df):
    
    '''En esta imputaremos los valores de \\n en el flag de centrales externas como 0 bajo la premisa que siempre
    tomamos y por ahora eliminaremos los individuos que no me reporten calificacion (solo son 9%) y no veo como
    imputarles un valor valido. Maybe la moda, pero es de discutirse'''
    
    df.pol_centr_ext = df.pol_centr_ext.replace(to_replace = ['foo', '\\N', float('nan')], value = ['0', '0', '0'])
    
    #df.rep_calif_cred= df.iloc[df.loc[(df.rep_calif_cred!='SIN INFORMACION')&(df.rep_calif_cred!='foo')].index,-4]
    moda = df.rep_calif_cred.mode().iloc[0]
    df.rep_calif_cred = df.pol_centr_ext.replace(to_replace = ['foo', 'SIN INFORMACION', float('nan')], value = [moda, moda, moda])
    df.pol_centr_ext=pd.to_numeric(df.pol_centr_ext)
    
    return df

In [4]:
header = open('header.txt', 'r')
header = header.read()
header = header.split(',')
header.append('col1')
header.append('col2')

dtype_inicial = {i: 'object' for i in header}

In [5]:
dask_df = dd.read_csv('Dataton_train.csv', names = header, dtype = dtype_inicial) #declaramos el dask dataframe 

In [6]:
dask_df = dask_df.map_partitions(fuck_comas) #implimentamos la correccion del desplazamiento de columnas

### Razones para eliminar variables

* id_cli no es necesario conocerlo
* fecha_nacimiento para eso se tiene la edad
* ciudad_residencia y ciudad_laboral porque son muchas categorias, se prefiere trabajar solo departamento
* tenencia_tc porque es un flag que surge de cupo_total_tc
* tiene_ctas_activas es redundante con ctas_activas
* tiene_ctas_embargadas es redundante con ctas_embargadas
* por ahora nos soplamos los departamentos para que cuando agreguemos doomies no sea un chicharron
* tambien nos soplamos el CIUU, eso no agrega varianza ni siquiera con la propuesta de reducir categorias
* Convenio libranza tampoco aggrega varianza. A pesar de tener mas de 20 valores unicos el 85% de veces es "\\N" o "REVISAR CONVENIO"

In [7]:
#Drop a priori de variables
dask_df = dask_df.drop(columns = ['id_cli', 'fecha_nacimiento', 'ciudad_residencia',
                                 'ciudad_laboral', 'tenencia_tc', 'tiene_ctas_activas', 
                                 'tiene_ctas_embargadas','departamento_residencia', 'departamento_laboral',
                                 'codigo_ciiu','convenio_lib'])

A fin de evitar los foo lo que haremos es aplicar a un df (df0 en este caso) la funcion antes que a las otras particiones y asi en los metadatos de diremos a Dask que clase de output esperar y desaparecen los foo

In [8]:
df0=dask_df.get_partition(0).compute()
df0=t_ult_actual_y_covid(df0)
dask_df = dask_df.map_partitions(t_ult_actual_y_covid,meta=df0)
df0=edad_to_float(df0)
dask_df = dask_df.map_partitions(edad_to_float,meta=df0)
df0=genero_limpio(df0)
dask_df = dask_df.map_partitions(genero_limpio,meta=df0)
df0=estado_civil_limpio(df0)
dask_df = dask_df.map_partitions(estado_civil_limpio,meta=df0)
df0=nivel_academico_limpio(df0)
dask_df = dask_df.map_partitions(nivel_academico_limpio,meta=df0)
df0=profesion_limpia(df0)
dask_df = dask_df.map_partitions(profesion_limpia,meta=df0)
df0=ocupacion_limpia(df0)
dask_df = dask_df.map_partitions(ocupacion_limpia,meta=df0)
df0=tipo_vivienda_limpia(df0)
dask_df = dask_df.map_partitions(tipo_vivienda_limpia,meta=df0)
df0=categoria_limpio(df0)
dask_df = dask_df.map_partitions(categoria_limpio,meta=df0)
df0=ind_binarios_riesgo(df0)
dask_df = dask_df.map_partitions(ind_binarios_riesgo,meta=df0)
df0=rechazo_credito_limpio(df0)
dask_df = dask_df.map_partitions(rechazo_credito_limpio,meta=df0)
df0=manejo_de_moras(df0)
dask_df = dask_df.map_partitions(manejo_de_moras,meta=df0)
df0=limpieza_TdC(df0)
dask_df = dask_df.map_partitions(limpieza_TdC,meta=df0)
df0=limpieza_obligaciones_cred(df0)
dask_df = dask_df.map_partitions(limpieza_obligaciones_cred,meta=df0)
df0=limpieza_cuentas(df0)
dask_df = dask_df.map_partitions(limpieza_cuentas,meta=df0)
df0=limpieza_fopep(df0)
dask_df = dask_df.map_partitions(limpieza_fopep,meta=df0)
df0=limpieza_hipotecario(df0)
dask_df = dask_df.map_partitions(limpieza_hipotecario,meta=df0)
df0=limpieza_mediana_nom3(df0)
dask_df = dask_df.map_partitions(limpieza_mediana_nom3,meta=df0)
df0=limpieza_mediana_pen3(df0)
dask_df = dask_df.map_partitions(limpieza_mediana_pen3,meta=df0)
df0=limpieza_ingreso_nompen(df0)
dask_df = dask_df.map_partitions(limpieza_ingreso_nompen,meta=df0)
df0=limpieza_cat_ingreso(df0)
dask_df = dask_df.map_partitions(limpieza_cat_ingreso,meta=df0)
df0=limpeza_al_estilo_datacredito(df0)
dask_df = dask_df.map_partitions(limpeza_al_estilo_datacredito,meta=df0)
df0=limpeza_ingresos_gastos(df0)
dask_df = dask_df.map_partitions(limpeza_ingresos_gastos,meta=df0)
df0=arreglo_col_mes(df0)
dask_df = dask_df.map_partitions(arreglo_col_mes,meta=df0)
df0=Limpieza_calif_cent_ext(df0)
dask_df = dask_df.map_partitions(Limpieza_calif_cent_ext,meta=df0)

In [ ]:
df0 = dask_df.get_partition(0)
df0 = df0.compute()
df0.columns

### Importante


+ las imputaciones se podrian hacer con el MICE https://www.statsmodels.org/stable/generated/statsmodels.imputation.mice.MICEData.html que propuso Ortiz

#### Explicacion de donde viene el foo
Indeed, if you read the docs for apply, you will see that meta= is a parameter that you can pass, which tells Dask how to expect the output of the operation to look. This is necessary because apply can do very general things.

If you don't supply meta=, as in your case, than Dask will try to seed the operation with an example mini-dataframe containing 1 for any numerical columns and "foo" for text ones, just to see what the output will be like. Since in your apply you print (and don't actually return anything), you are seeing this seed.

As suggested by the documentation, you are always better off providing meta= when possible, and then a whole step in the process can be avoided.
    

## Análisis exploratorio de datos

### Funciones

In [9]:
def histogramas(df0, variable):
    
    '''Esta funcion grafica los histogramas de la variable elegida. Imprime uno con todos los datos y otro
    con los datos filtrados para el gasto familiar mayor a cero y menor a 2 millones.
    recibe el dataframe de los datos y el string del nombre de la variable'''
    
    fig, ax = plt.subplots(1, 1,figsize=(13, 6))

    plt.hist(data = df0, x = variable, density = False, bins = 'doane');
    plt.xlabel(variable);
    plt.title('Histograma de: ' + variable);

    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.xaxis.set_major_formatter(tick) 

    fig, ax = plt.subplots(1, 1,figsize=(13, 6))

    plt.hist(data = df0[(df0['gasto_familiar'] > 0) & (df0['gasto_familiar'] < 2000000)], x = variable, density = False, bins = 'doane');
    plt.xlabel(variable);
    plt.title('Histograma de: ' + variable + " zoom in entre 0 COP y 2M COP para el gasto familiar");

    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.xaxis.set_major_formatter(tick) 

def percentiles(df0, variable):
    
    '''Grafica un grafico de pencentiles para la variable seleccionada y hace zoom in para los 
    que esten entre el percentil 10 y el 90'''
    
    fig, ax = plt.subplots(1, 1,figsize=(8, 5))

    percentiles = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    valores = np.percentile(a = df0[variable], q = percentiles)

    plt.plot(percentiles, valores);
    plt.xlabel('Percentil');
    plt.ylabel(variable);
    plt.title(variable + ' por percentiles');

    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick) 


    fig, ax = plt.subplots(1, 1,figsize=(8, 5))

    percentiles = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    valores = np.percentile(a = df0[variable], q = percentiles)

    plt.plot(percentiles, valores);
    plt.xlabel('Percentil');
    plt.ylabel(variable);
    plt.title(variable + ' por percentiles zoom in entre 10% y 90%');

    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick) 

def box_violin(df0, x, y, tipo = 'box'):
    
    '''Imprime los boxplot o violinplots de la variable 'y' vs el factor 'x'. Tambien imprime los mismo
    pero filtrando por gasto familiar entre 0 y 2 millones'''
    
    if tipo == 'box':

        fig, ax = plt.subplots(1, 1, figsize=(13, 5))

        sns.boxplot(x, y, data = df0, orient = 'v');
        plt.xlabel(x);
        plt.ylabel(y);
        plt.title('Boxplot por ' + x + ' del ' + y)

        fmt = '{x:,.0f}'
        tick = mtick.StrMethodFormatter(fmt)
        ax.yaxis.set_major_formatter(tick)


        fig, ax = plt.subplots(1, 1,figsize=(13, 5))

        sns.boxplot(x, y, data = df0[(df0['gasto_familiar'] > 0) & (df0['gasto_familiar'] < 2000000)], orient = 'v');
        plt.xlabel(x);
        plt.ylabel(y);
        plt.title('Boxplot por ' + x + ' del ' + y + ' con zoom in entre 0 COP y 2M COP')

        fmt = '{x:,.0f}'
        tick = mtick.StrMethodFormatter(fmt)
        ax.yaxis.set_major_formatter(tick) 
        
    elif tipo == 'violin':
        
        fig, ax = plt.subplots(1, 1, figsize=(13, 5))

        sns.violinplot(x, y, data = df0, orient = 'v');
        plt.xlabel(x);
        plt.ylabel(y);
        plt.title('Violinplot por ' + x + ' del ' + y)

        fmt = '{x:,.0f}'
        tick = mtick.StrMethodFormatter(fmt)
        ax.yaxis.set_major_formatter(tick)


        fig, ax = plt.subplots(1, 1,figsize=(13, 5))

        sns.violinplot(x, y, data = df0[(df0['gasto_familiar'] > 0) & (df0['gasto_familiar'] < 2000000)], orient = 'v');
        plt.xlabel(x);
        plt.ylabel(y);
        plt.title('Violinplot por ' + x + ' del ' + y + ' con zoom in entre 0 COP y 2M COP')

        fmt = '{x:,.0f}'
        tick = mtick.StrMethodFormatter(fmt)
        ax.yaxis.set_major_formatter(tick) 
    
def scatter(df0, x, y):   

    '''Diagrama de dipersion de la variable y vs la variable x. Obtiene lo mismo para los datos filtrados con 
    ingreso familiar entre 0 COP y 2 millones. A la y le aplica transformacion logaritmica'''

    fig, ax = plt.subplots(1, 1,figsize=(13, 5));
    plt.scatter(x, y, data = df0);
    plt.xlabel(x);
    plt.ylabel(y);
    plt.title('Scatterplot de ' + y + ' vs ' + x);
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick) 
    ax.xaxis.set_major_formatter(tick)

    fig, ax = plt.subplots(1, 1, figsize=(13, 5));
    plt.scatter(x, y, data = df0[(df0['gasto_familiar'] > 0) & (df0['gasto_familiar'] < 2000000)]);
    plt.xlabel(x);
    plt.ylabel(y);
    plt.title('Scatterplot de ' + y + ' vs ' + x + ' con zoom in entre 0 COP y 2M COP');
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick) 
    ax.xaxis.set_major_formatter(tick)
    
def correlacion(df0, x, y):
    
    '''Calcula tres coeficientes de correlacion para las variables x e y. 
    Aplica lo mismo filtrando para gasto_familiar entre 0 COP y 2 millones'''
    
    print('Correlaciones para: ' + x + ' vs ' + y + ':' + '\n')
    
    pearson_sin_limpiar = df0[x].corr(df0[y], method = 'pearson') 
    spearmann_sin_limpiar = df0[x].corr(df0[y], method = 'spearman') 
    kendall_sin_limpiar = df0[x].corr(df0[y], method = 'kendall') 
    
    print('Pearson sin limpiar: ' + str(pearson_sin_limpiar)[:6])
    print('Spearmann sin limpiar: ' + str(spearmann_sin_limpiar)[:6])
    print('Kendall sin limpiar: ' + str(kendall_sin_limpiar)[:6] + '\n')
    
    df0_filtrado = df0[(df0['gasto_familiar'] > 0) & (df0['gasto_familiar'] < 2000000)]
    
    pearson_filtrado = df0_filtrado[x].corr(df0_filtrado[y], method = 'pearson') 
    spearmann_filtrado = df0_filtrado[x].corr(df0_filtrado[y], method = 'spearman') 
    kendall_filtrado = df0_filtrado[x].corr(df0_filtrado[y], method = 'kendall') 
    
    print('Pearson filtrado: ' + str(pearson_filtrado)[:6])
    print('Spearmann filtrado: ' + str(spearmann_filtrado)[:6])
    print('Kendall filtrado: ' + str(kendall_filtrado)[:6])

In [ ]:
histogramas(df0, 'gasto_familiar')

In [ ]:
percentiles(df0, 'gasto_familiar')

In [ ]:
box_violin(df0, 'rep_calif_cred', 'gasto_familiar', 'box')

In [ ]:
scatter(df0, 'ind', 'gasto_familiar')

In [ ]:
correlacion(df0, 'ind', 'gasto_familiar')

## Finalización del preprocesamiento

Con las funciones anteriores, se analizó variable por variable analizando si agregaba o no agregaba varianza. De manera que se pudiese elegir cuales variables merecen ser eliminadas y cuales tienen el beneficio de la deuda.

Ahora se retiran las variables que no afectan la variable de respuesta. Revisar word con el nombre 'limpieza a partir del eda.docx' para verla categorizacion de las variables

In [10]:
#Drop con el EDA de variables
dask_df = dask_df.drop(columns = ['mes', 'covid', 'edad', 'genero', 
                                 't_ult_actual', 'ind_mora_vigente', 'cartera_castigada', 
                                 'cupo_total_tc', 'cuota_tc_bancolombia', 'tiene_consumo', 
                                 'pension_fopep', 'mediana_nom3', 'mediana_pen3', 
                                'ingreso_nompen','cuota_de_consumo', 'cuota_rotativos',
                                  'cuota_tarjeta_de_credito', 'cuota_de_sector_solidario',
                                  'cuota_sector_real_comercio', 'saldo_prom3_tdc_mdo',
                                  'saldo_no_rot_mdo', 'cuota_libranza_sf'])

In [ ]:
df0 = dask_df.get_partition(0)

In [ ]:
df0 = df0.compute()

In [ ]:
df0.head()

In [ ]:
df0.columns

Ahora se convierten a numerico las variables que no quedaron como tal en el preprocesamiento anterior y se obtienen las dummies

In [11]:
def finalizar_preprocesamiento(df0):
    
    '''Funcion que obtiene los dummies, transforma a numerico las que no quedaron como tal y 
    lleva la variable gasto al final del dataframe'''
    
    
    #Obtener dummies
    list_dummies = ['estado_civil', 'nivel_academico', 'ocupacion', 'tipo_vivienda', 'cat_ingreso']
    
    for i in list_dummies:
        var_dummies = pd.get_dummies(df0[i])
        df0.drop(columns = [i], inplace = True)
        df0 = pd.concat([df0, var_dummies], axis = 1)
        
    #Transformar a numerico
    
    #rep_calif_cred es cuestionable convertirla a numerico. Hay que debatirlo
    to_numericas = ['categoria', 'tiene_cred_hipo_1', 'tiene_cred_hipo_2', 'rep_calif_cred']
    
    for i in to_numericas:
        #df0[i] = df0[i].replace(to_replace = ['foo'], value = [float('nan')])
        df0[i] = [float(j) for j in df0[i]]
    
    
    #llevar gasto_familiar al final
    cols = df0.columns.tolist()
    cols = cols[-36:] + cols[:-36]
    df0 = df0[cols]
    
    return df0

In [12]:
df0 = dask_df.get_partition(0)
df0 = df0.compute()
df0 = finalizar_preprocesamiento(df0)
dask_df = dask_df.map_partitions(finalizar_preprocesamiento,meta=df0) #el foo cagaba todo

In [ ]:
dask_df

In [ ]:
df0.head()

In [ ]:
#aca volvemos a sacar el Df0 del dask df para comprobar que todo aplico bien
df0 = dask_df.get_partition(0)
df0 = df0.compute()
df0.head()

## Obtener los registros que se deben predecir y otros concernientes a la finalización del preprocesamiento

El siguiente desarrollo permite obtener los registros que se deben predecir, ya preprocesados. 

In [13]:
def obtener_X_test():  
    
    '''La siguiente funcion preprocesa el data set a predecir y devuelve una tupa con X_test y otra con id_registro'''
    
    header = open('header.txt', 'r')
    header = header.read()
    header = header.split(',')
    header.append('col1')
    header.append('col2')

    dtype_inicial = {i: 'object' for i in header}

    header.insert(0, 'id_registro')

    bd_test = dd.read_csv('dt2020_base_evaluar.csv', names = header, dtype = dtype_inicial)

    bd_test = bd_test.map_partitions(fuck_comas) #implimentamos la correccion del desplazamiento de columnas

    bd_test = bd_test.map_partitions(t_ult_actual_y_covid)
    bd_test = bd_test.map_partitions(edad_to_float) 
    bd_test = bd_test.map_partitions(genero_limpio)
    bd_test = bd_test.map_partitions(estado_civil_limpio)
    bd_test = bd_test.map_partitions(nivel_academico_limpio)
    bd_test = bd_test.map_partitions(profesion_limpia)
    bd_test = bd_test.map_partitions(ocupacion_limpia)
    bd_test = bd_test.map_partitions(tipo_vivienda_limpia)
    bd_test = bd_test.map_partitions(categoria_limpio)
    bd_test = bd_test.map_partitions(ind_binarios_riesgo)
    bd_test = bd_test.map_partitions(rechazo_credito_limpio)
    bd_test = bd_test.map_partitions(manejo_de_moras)
    bd_test = bd_test.map_partitions(limpieza_TdC)
    bd_test = bd_test.map_partitions(limpieza_obligaciones_cred)
    bd_test = bd_test.map_partitions(limpieza_cuentas)
    bd_test = bd_test.map_partitions(limpieza_fopep)
    bd_test = bd_test.map_partitions(limpieza_hipotecario)
    bd_test = bd_test.map_partitions(limpieza_mediana_nom3)
    bd_test = bd_test.map_partitions(limpieza_mediana_pen3)
    bd_test = bd_test.map_partitions(limpieza_ingreso_nompen)
    bd_test = bd_test.map_partitions(limpieza_cat_ingreso)
    bd_test = bd_test.map_partitions(limpeza_al_estilo_datacredito)
    bd_test = bd_test.map_partitions(limpeza_ingresos_gastos)
    bd_test = bd_test.map_partitions(arreglo_col_mes)
    bd_test = bd_test.map_partitions(Limpieza_calif_cent_ext)

    bd_test = bd_test.drop(columns = ['id_cli', 'fecha_nacimiento', 'ciudad_residencia',
                                     'ciudad_laboral', 'tenencia_tc', 'tiene_ctas_activas', 
                                     'tiene_ctas_embargadas','departamento_residencia', 'departamento_laboral',
                                     'codigo_ciiu','convenio_lib'])

    bd_test = bd_test.drop(columns = ['mes', 'covid', 'edad', 'genero', 
                                     't_ult_actual', 'ind_mora_vigente', 'cartera_castigada', 
                                     'cupo_total_tc', 'cuota_tc_bancolombia', 'tiene_consumo', 
                                     'pension_fopep', 'mediana_nom3', 'mediana_pen3', 
                                    'ingreso_nompen','cuota_de_consumo', 'cuota_rotativos',
                                      'cuota_tarjeta_de_credito', 'cuota_de_sector_solidario',
                                      'cuota_sector_real_comercio', 'saldo_prom3_tdc_mdo',
                                      'saldo_no_rot_mdo', 'cuota_libranza_sf'])

    bd_test_0 = bd_test.get_partition(0)
    bd_test_0 = bd_test_0.compute()
    #bd_test_0.head()

    bd_test_1 = bd_test.get_partition(1)
    bd_test_1 = bd_test_1.compute()
    #bd_test_1.head()

    bd_test_2 = bd_test.get_partition(2)
    bd_test_2 = bd_test_2.compute()
    #bd_test_2.head()

    X_test = pd.concat([bd_test_0, bd_test_1, bd_test_2], axis = 0)
    X_test.drop(columns = ['gasto_familiar'], inplace = True)
    id_registro = X_test['id_registro']
    X_test.drop(columns = ['id_registro'], inplace = True)

    #Obtener dummies
    list_dummies = ['estado_civil', 'nivel_academico', 'ocupacion', 'tipo_vivienda', 'cat_ingreso']

    for i in list_dummies:
        var_dummies = pd.get_dummies(X_test[i])
        X_test.drop(columns = [i], inplace = True)
        X_test = pd.concat([X_test, var_dummies], axis = 1)

    #Transformar a numerico

    #rep_calif_cred es cuestionable convertirla a numerico. Hay que debatirlo
    to_numericas = ['categoria', 'tiene_cred_hipo_1', 'tiene_cred_hipo_2', 'rep_calif_cred']

    for i in to_numericas:
        #df0[i] = df0[i].replace(to_replace = ['foo'], value = [float('nan')])
        X_test[i] = [float(j) for j in X_test[i]]

    #llevar gasto_familiar al final
    cols = X_test.columns.tolist()
    cols = cols[-36:] + cols[:-36]
    X_test = X_test[cols]

    return (X_test, id_registro)

In [14]:
X_test, id_registro = obtener_X_test()

In [ ]:
X_test.head()

In [ ]:
id_registro

La siguiente funcion es para imprimir los resultados en formato csv

In [15]:
def imprimir_resultado(y_test_pred, nombre_archivo):
    
    '''Funcion que imprime el resultado, al nombre del archivo debe agregarse un .csv'''
    
    data = {'id_registro': id_registro, 'gasto_familiar': y_test_pred}
    
    output = pd.DataFrame(data)
    output = output.set_index('id_registro')
    output.to_csv(nombre_archivo)

La siguiente funcion es para obtener las primeras ### particiones del dask, pronto será deprecated

In [16]:
def primeras_particiones(k, atipicos):
    
    '''Entrega una tupla con X e y del dask, indicando las primeras k particiones a tomar en cuenta. 
    Entrega el resultado filtrado con abs(Z) > 2.7 or gastos_familiar_{i} negativos si atipicos = 'Z' o solo 
    lo de los negativos si atipicos = 'neg'. 
    Siendo Z = (xi - med)/mad'''
    
    part_7dddf = dask_df.get_partition(0).compute()
    
    for i in range(1, k):
        part_7dddf = pd.concat([part_7dddf,dask_df.get_partition(i).compute()], axis = 0)
        
    X = part_7dddf.iloc[:, 0:60] 
    y = part_7dddf["gasto_familiar"]
    
    med = part_7dddf['gasto_familiar'].median()
    mad = median_absolute_deviation(part_7dddf['gasto_familiar'])
    
    if atipicos == 'Z':
    
        indicador = [False if (abs(i-med)/mad) > 2.7 or i < 0 else True for i in part_7dddf['gasto_familiar']]
        
    elif atipicos == 'neg':
        
        indicador = [False if i < 0 else True for i in part_7dddf['gasto_familiar']]
        
    
    part_7dddf = part_7dddf.iloc[indicador]
    X = X.iloc[indicador] 
    y = y.iloc[indicador]
    
    return (X, y)

In [17]:
X, y = primeras_particiones(20, atipicos = 'neg')

La siguiente funcion calcula el MAPE

In [18]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

## Reducción de dimensiones, cluster y clasificador

El siguiente código aplica PCA a los datasets de X y X_test

In [ ]:
#se declara el escalador
scaler = StandardScaler()

#se entrena el escalador
scaler.fit(X)

#se escalan los datos
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

#se declara el PCA
pca = PCA(.95)

#se entrena el PCA
pca.fit(X_scaled)

#se transforman los datos escalados a PCA's
X_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

#se renombran las variables

X = X_pca
X_test = X_test_pca


El siguiente clustering para los datos, notese que no funciono, como lo muestran las graficas de inercia. Primero se hizo hasta k=20 y luego hasta k=40, pero si se hace para más, el codo seguira siendo impercentible

In [ ]:
#esta es la inercia para un kmeans corrido desde k=2 hasta k=39

#for i in range (20, 40):
#    kmeans = KMeans(n_clusters = i, random_state = 0).fit(X)
#    inertia.append(kmeans.inertia_)

inertia = [77112762.79291268, 74331591.13173904, 71957295.60106312, 70071725.40839572, 66745896.11933103, 65267013.742731795,
           63789473.7435838, 62288995.86606553, 60501387.813032605, 59038854.92246048, 57651711.06445355, 56340192.00861995,
           54955176.61438507, 53557538.35161901, 52610487.46659651, 51214593.92215118, 49845423.85100328, 48777525.3523114, 
           47960946.72825996, 46191257.500313915, 45725034.43820399, 44757639.20487177, 43892272.63960439, 42619398.43980851, 
           42071740.27109539, 41179808.436337605, 39862378.28405561, 38798079.69976133, 38398742.84839752, 37679648.0023016, 
           36659210.332658805, 35872029.634098604, 35347221.73675595,34958985.8747368, 34538836.417099416, 33908841.54218117, 
           33375833.292873614, 32743859.728969246]

plt.plot(range(2, 40), inertia);


In [ ]:
pd.DataFrame([(inertia[i-1] - inertia[i])*100/inertia[i-1] for i in range(1, len(inertia))]).plot();

# Ahora aca si estamos haciendo bien la regla del codo para clustering

XD

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,30), timings= True)
visualizer.fit(X)        # Fit data to visualizer
visualizer.show()    

In [ ]:
visualizer = KElbowVisualizer(model, k=(30,60), timings= True)
visualizer.fit(X)        # Fit data to visualizer
visualizer.show()    

In [ ]:
visualizer = KElbowVisualizer(model, k=(60,80), timings= True)
visualizer.fit(X)        # Fit data to visualizer
visualizer.show()    

# Aqui comienza la clusterizacion ideal

In [ ]:
kmeans = KMeans(n_clusters = 46, random_state = 0).fit(X)

In [ ]:
logistica = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
logistica.fit(X, kmeans.labels_)
logistica.score(X, kmeans.labels_)

In [ ]:
np.sort(pd.Series(kmeans.labels_).unique())

Para que funcione mi logica necesito X y Y como dataframes de nuevo asi que los convierto

In [ ]:
X=pd.DataFrame(X)
y=pd.DataFrame(y)
X["label"]=kmeans.labels_
y["label"]=kmeans.labels_

Despues de entrenar la hija de puta logistica tanto tiempo vamos a entrenar todas las ridge

In [ ]:
ridges=[Ridge(alpha = 0.00001, normalize=True) for i in range(46)]
for i in range(46):
    ridges[i].fit(X.iloc[X.loc[X["label"]==i].index,:-1],y.iloc[y.loc[y["label"]==i].index,:-1])


Ahora clasificaremos los datos de test y le sacamos de una vez las predicciones en orden.
Debe haber una mejor manera de sacarla que no involucre la list comprehension, pero no se me ocurrió como para
que siguiera en orden

In [ ]:
labels_test=logistica.predict(X_test)
X_test=pd.DataFrame(X_test)
X_test['label']=labels_test
por el metodo iloc toca darle una forma optima para que la regresion pueda leer bien los datos
# aca nos figuro darle la forma apropiada a cada set de datos de x para que el predict lo leyera
y_test_pred=[ridges[X_test.iloc[i,-1]].predict(np.asarray(X_test.iloc[i,:-1]).reshape(1, 43)) for i in range(len(X_test))]
# aqui lo volvemos una lista de integers y no de arrays
y_test_pred=[item for i in y_test_pred for item in i.tolist ]



In [ ]:
y_test_pred

In [ ]:
imprimir_resultado(y_test_pred,"The Data Machinery Submission 28.csv")

## Clasificador de ricos, medios y pobres con Reg Ridge

Dejemos a un lado el enfoque de cluster y pasemos al de clasificacion

In [19]:
P95 = np.percentile(a = y, q = np.array([95]))
etiqueta = ['Rico' if i >= P95 else 'No Rico' for i in y]

In [20]:
logistica = LogisticRegression()

In [21]:
logistica.fit(X, etiqueta)

LogisticRegression()

In [22]:
logistica.score(X, etiqueta)

0.9455631948019545

La logística anterior predice muy bien, aparentemente. 

In [23]:
Ricos = [True if i == 'Rico' else False for i in etiqueta]  
No_ricos = [True if i == 'No Rico' else False for i in etiqueta]  

In [24]:
X_ricos = pd.DataFrame(X).iloc[Ricos]
y_ricos = pd.DataFrame(y).iloc[Ricos]

In [25]:
X_no_ricos = pd.DataFrame(X).iloc[No_ricos]
y_no_ricos = pd.DataFrame(y).iloc[No_ricos]

In [26]:
len(X_ricos)

142285

In [27]:
len(X_no_ricos)

2703400

In [28]:
ridge_ricos = Ridge(alpha = 0.00001, normalize=True)
ridge_no_ricos = Ridge(alpha = 0.00001, normalize=True)

In [29]:
ridge_ricos.fit(X_ricos, y_ricos)
ridge_no_ricos.fit(X_no_ricos, y_no_ricos)

Ridge(alpha=1e-05, normalize=True)

In [30]:
print("Mape de la ridge no ricos: " + str(mean_absolute_percentage_error(np.array(y_no_ricos) + 1000, ridge_no_ricos.predict(X_no_ricos))))

Mape de la ridge no ricos: 41.362660711146084


In [ ]:
scores = []

for i in range(10, 90, 5):
    
    Pi = np.percentile(a = y_no_ricos.gasto_familiar, q = np.array([i]))
    etiqueta_2 = ['Medio' if i>=Pi else 'Pobre' for i in y_no_ricos.gasto_familiar]
    
    logistica_2 = LogisticRegression()
    logistica_2.fit(X_no_ricos, etiqueta_2)
    scores.append(logistica_2.score(X_no_ricos, etiqueta_2))

In [ ]:
pd.DataFrame(scores).plot();
plt.vlines(2, ymin = 0.75, ymax = 0.85);

In [31]:
P20 = np.percentile(a = y_no_ricos.gasto_familiar, q = np.array([20]))
etiqueta_2 = ['Medio' if i>=P20 else 'Pobre' for i in y_no_ricos.gasto_familiar]

In [32]:
logistica_2 = LogisticRegression()
logistica_2.fit(X_no_ricos, etiqueta_2)
logistica_2.score(X_no_ricos, etiqueta_2)

0.7999178811866539

In [33]:
Medio = [True if i == 'Medio' else False for i in etiqueta_2]  
Pobre = [True if i == 'Pobre' else False for i in etiqueta_2]  

In [34]:
X_medio = pd.DataFrame(X_no_ricos).iloc[Medio]
y_medio = pd.DataFrame(y_no_ricos).iloc[Medio]

In [35]:
X_pobre = pd.DataFrame(X_no_ricos).iloc[Pobre]
y_pobre = pd.DataFrame(y_no_ricos).iloc[Pobre]

In [36]:
len(X_medio)

2162720

In [37]:
len(y_pobre)

540680

In [44]:
nulidad_pobre=["nulo" if i == 0 else "no_nulo" for i in y_pobre.gasto_familiar]
Logistica_de_pobres=LogisticRegression()
Logistica_de_pobres.fit(X_pobre,nulidad_pobre)
Logistica_de_pobres.score(X_pobre,nulidad_pobre)

0.674254642302286

In [45]:
nulidad_pobre_pred=Logistica_de_pobres.predict(X_test)
X_pobre['nulidad']=nulidad_pobre
pd.Series(nulidad_pobre_pred).value_counts()

no_nulo    256868
nulo        24798
dtype: int64

In [46]:
ridge_medio = Ridge(alpha = 0.00001, normalize = True)
ridge_pobre = Ridge(alpha = 0.00001, normalize = True)

In [47]:
ridge_medio.fit(X_medio, y_medio)
ridge_pobre.fit(X_pobre.iloc[X_pobre.loc[X_pobre.nulidad=="no_nulo"].index,:-1], y_pobre.iloc[X_pobre.loc[X_pobre.nulidad=="no_nulo"].index])

Ridge(alpha=1e-05, normalize=True)

In [50]:
print("Mape de la ridge medios: " + str(mean_absolute_percentage_error(np.array(y_medio) , ridge_ricos.predict(X_medio))))

Mape de la ridge medios: 11.998408017745597


In [53]:
print("Mape de la ridge pobres: " + str(mean_absolute_percentage_error(np.array(y_pobre.iloc[X_pobre.loc[X_pobre.nulidad=="no_nulo"].index]) + 1000, ridge_ricos.predict(X_pobre.iloc[X_pobre.loc[X_pobre.nulidad=="no_nulo"].index,:-1]))))

Mape de la ridge pobres: 1878.868067083032


In [54]:
etiqueta1_pred=logistica.predict(X_test)

In [59]:
ind_no_ricos=[True if i=='No Rico' else False for i in etiqueta1_pred]
X_test_no_ricos=X_test.iloc[ind_no_ricos,:]
etiqueta2_pred=logistica_2.predict(X_test_no_ricos)
pd.Series(etiqueta2_pred).value_counts()

Medio    281309
dtype: int64

In [71]:
y_pred_1=ridge_ricos.predict(X_test)
y_pred_1[ind_no_ricos]=ridge_medio.predict(X_test_no_ricos)

In [72]:
y_pred_1[ind_no_ricos]=0.5*y_pred_1[ind_no_ricos]
y_pred_1 = [item for i in y_pred_1 for item in i.tolist()]
imprimir_resultado(y_pred_1,"The Data Machinery submission 39.csv")

In [62]:
y_pred = [item for i in y_pred_1 for item in i.tolist()]
imprimir_resultado(y_pred,"The Data Machinery submission 37.csv")

In [ ]:
histogramas(y_pobre, 'gasto_familiar')

In [ ]:
percentiles(y_no_ricos, 'gasto_familiar')

Parece que no hay diferencias entre las variables diferentes a gasto_familiar entre las personas más pobres. Las cuales por cierto, tienen su gatso_familiar en cero. Haremos una regresion que no incluya a los de gasto familiar = 0

In [ ]:
y = pd.DataFrame(y)
X = pd.DataFrame(X)

In [ ]:
X = X.iloc[y.loc[y.gasto_familiar != 0].index]
y = y.iloc[y.loc[y.gasto_familiar != 0].index]

In [ ]:
ridge_sin_pobres_weon = Ridge(alpha = 0.00001, normalize=True)

In [ ]:
ridge_sin_pobres_weon.fit(X, y)

In [ ]:
print("Mape de la ridge completa sin ceros: " + str(mean_absolute_percentage_error(np.array(y) + 1000, ridge_sin_pobres_weon.predict(X))))

In [ ]:
y_pred = ridge_sin_pobres_weon.predict(X_test)

In [ ]:
y_pred = [item for i in y_pred for item in i.tolist()]

In [ ]:
imprimir_resultado(y_pred, 'The Data Machinery Submission 29.csv')

# Aca intentaremos clasificar a los raros que dan 0 en vez de meterlos a regresion

In [ ]:
etiqueta = ['nulo' if i == 0 else 'no_nulo' for i in y]

In [ ]:
pd.Series(pred_0).value_counts()

In [ ]:
#logistica_0=LogisticRegression()
#logistica_0.fit(X,etiqueta)
#logistica_0.score(X,etiqueta)

In [ ]:
pred_0=logistica_0.predict(X_test)

In [ ]:
X["nulidad"]=etiqueta
ridge_no_nulos=Ridge(alpha = 0.00001, normalize=True)
ridge_no_nulos.fit(X.iloc[X.loc[X.nulidad=="no_nulo"].index,:-1],y.iloc[X.loc[X.nulidad=="no_nulo"].index,])

In [ ]:
X_test["nulidad"]=pred_0
X_test.iloc[X_test.loc[X_test.nulidad=="no_nulo"].index,:-1]
y_test=ridge_no_nulos.predict(X_test.iloc[:,:-1])

In [ ]:
y_pred=ridge_medio.predict(X_test)

In [ ]:
y_pred = [item for i in y_pred for item in i.tolist()]

In [ ]:
imprimir_resultado(y_pred,"The Data Machinery Submission 35.csv")

# Con X_ricos y X_no_ricos logré dos Ridge que tienen 0.5% y 40% de MAPE puede implementarse otra oarticion a los no_ricos para sacar medios y pobresy asi entrenar tres regresiones. pipe haz lo tuyo

# Pendientes pre modelamiento

+ la transformacion de rep_calif_cred de object a float es muy debatible, hay que revisarlo

+ integrar R para probar el kurtosis projections

+ esperar resultados de Ortiz al ejecutar en R el codigo

+ Implementar PCA's para disminuir dimensiones y así mejorar la velocidad de ajuste de los modelos https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60


# Ajuste de modelos

### Modelamiento breve con Random Forest

In [ ]:
random_forest = RandomForestRegressor(n_estimators = 6, max_depth = 2,  random_state = 0)

In [ ]:
random_forest.fit(X, y)

In [ ]:
print("Mape_train_general: " + str(mean_absolute_percentage_error(np.array(y) + 1000, random_forest.predict(X))))

In [ ]:
imprimir_resultado(random_forest.predict(X_test), 'ensayo mil con random forest.csv')

### Modelamiento super breve con RIDGE

In [ ]:
reg_ridge = Ridge(alpha = 0.00001, normalize=True)

In [ ]:
reg_ridge.fit(X_no_ricos, y_no_ricos)

In [ ]:
print("Mape_train_general: " + str(mean_absolute_percentage_error(np.array(y_no_ricos) + 1000, reg_ridge.predict(X_no_ricos))))

In [ ]:
imprimir_resultado(reg_ridge.predict(X_test), 'The Data Machinery Submission 27.csv')

### Modelamiento breve con SVM Regressor ese hpta es muy lento, lo corre su madre

In [ ]:
#svr_model = SVR(kernel = 'linear')

In [ ]:
#svr_model.fit(X, y)

In [ ]:
#print("Mape_train_general: " + str(mean_absolute_percentage_error(np.array(y)+1000, svr_model.predict(X))))

In [ ]:
#y_test_pred = svr_model.predict(X_test)

### Modelamiento breve con KNN Regressor

In [ ]:
knn_regressor = KNeighborsRegressor(n_neighbors = 300)
knn_regressor.fit(X, y)

In [ ]:
print("Mape_train_general: " + str(mean_absolute_percentage_error(np.array(y)+1000, knn_regressor.predict(X))))

In [ ]:
y_test_pred = knn_regressor.predict(X_test)

In [ ]:
imprimir_resultado(y_test_pred, 'ensayo tres mil con KNN.csv')

### Modelamiento breve con Gradient Boosting

In [ ]:
gradient_boosting = GradientBoostingRegressor(n_estimators = 10, max_depth = 3, learning_rate = 0.1, random_state = 0)

In [ ]:
gradient_boosting.fit(X, y)

In [ ]:
print("Mape_train_general: " + str(mean_absolute_percentage_error(np.array(y)+1000, gradient_boosting.predict(X))))

# Laboratorio de Paralelismo en métodos de predicción

Aca abajo intentamso corre un gradient boosting de dask con xgboost pero nos cagó el tener variables categoricas

In [ ]:
import dask.distributed
import xgboost as xgb


In [ ]:
data = {'id_registro':id_registro, 'gasto_familiar':y_test_pred}
output = pd.DataFrame(data)
output = output.set_index('id_registro')
output.to_csv('The Data Machinery Submission 10.csv')

In [ ]:
cluster = dask.distributed.LocalCluster(n_workers=8, threads_per_worker=4)
client = dask.distributed.Client(cluster)

In [ ]:
X = dask_df.iloc[:, 0:60] 
y = dask_df["gasto_familiar"] #la variable a predecir
#dtrain = xgb.dask.DaskDMatrix(client, X, y,enable_categorical=True)

In [ ]:
output = xgb.dask.train(client,
                        {'verbosity': 2,
                         'tree_method': 'hist',
                         'objective': 'reg:squarederror'
                         },
                        dtrain,
                        num_boost_round=4, evals=[(dtrain, 'train')])

Aca intentamos correr un randomforest con muchos umpaloompas

In [ ]:
from dask.distributed import Client
import joblib

In [ ]:
#client = Client(processes=False)             # create local cluster
# client = Client("scheduler-address:8786")  # or connect to remote cluster
random_forest = RandomForestRegressor(n_estimators = 150, max_depth = 5,  random_state = 0)
with joblib.parallel_backend('dask'):
    random_forest.fit(X, y)    


In [ ]:
client.close()